# Part A: Prepare a 🤗 Hugging Face Dataset from Data from CSV Format

In [1]:
!pip install datasets transformers shutup ipython-autotime evaluate shutup --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.7 MB/s eta 0:00:00


In [2]:
%load_ext autotime
import shutup
shutup.please()

time: 10.8 ms (started: 2023-01-22 22:09:11 +00:00)


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

pd.set_option('max_colwidth', 400)

time: 2.27 s (started: 2023-01-22 22:09:11 +00:00)


In [4]:
dataset_file = 'custom.csv'
base_path = '/content/drive/MyDrive/Machine_Learning/Transformers-Classification/'

time: 567 µs (started: 2023-01-22 22:09:13 +00:00)


In [5]:
dataset = load_dataset("csv", data_files=base_path+dataset_file)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4da75f7747cf2cc0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

time: 2.86 s (started: 2023-01-22 22:09:13 +00:00)


In [6]:
dataset = dataset["train"].shuffle(seed=42)

time: 13.4 ms (started: 2023-01-22 22:09:16 +00:00)


In [7]:
dataset = dataset.rename_column(
    original_column_name="labels", new_column_name="label"
)
# dataset = dataset.remove_columns('labels')

time: 7.01 ms (started: 2023-01-22 22:09:16 +00:00)


In [8]:
dataset = dataset.train_test_split(train_size=0.9, seed=42)
# Rename the default "test" split to "validation"
dataset["validation"] = dataset.pop("test")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 15311
    })
    validation: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 1702
    })
})

time: 51.2 ms (started: 2023-01-22 22:09:16 +00:00)


In [9]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [10]:
dataset.push_to_hub("dipesh/Intent-Classification-Commands-balanced-ds-large")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

time: 10.1 s (started: 2023-01-22 22:09:35 +00:00)


# PART B: Hugging Face Tokenization: Use a Pre-trained Tokenizer or Train a New Tokenizer from scratch?

As suggested in the official documentation, if a language model is not available in the language you are interested in, or if your corpus is very different from the one your language model was trained on, you will most likely want to retrain the model from scratch using a tokenizer adapted to your data. That will require training a new tokenizer on your dataset.

## Using Pretrained

In [11]:
base_model = 'joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english'

time: 632 µs (started: 2023-01-22 22:09:45 +00:00)


In [12]:
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)

outputs = pretrained_tokenizer(
    dataset["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=False,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print("vocab_size: ", len(pretrained_tokenizer))

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Input IDs length: 2
Input chunk lengths: [7, 10]
vocab_size:  30522
time: 12.8 s (started: 2023-01-22 22:09:47 +00:00)


In [13]:
txt = "how are you"
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)

[101, 2129, 2024, 2017, 102]
time: 2.13 ms (started: 2023-01-22 22:13:21 +00:00)


In [14]:
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 541 µs (started: 2023-01-22 22:13:25 +00:00)


## Train a new tokenizer

In [15]:
def get_training_corpus():
    batch_size = 1000
    return (
        dataset["train"][i : i + batch_size]["text"]
        for i in range(0, len(dataset["train"]), batch_size)
    )
training_corpus = get_training_corpus()

time: 1.03 ms (started: 2023-01-22 22:13:31 +00:00)


In [16]:
for data in get_training_corpus():
    print(len(data))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
311
time: 140 ms (started: 2023-01-22 22:13:32 +00:00)


### NOTE: Now that we have our corpus in the form of an iterator of batches of texts, we are ready to train a new tokenizer. To do this, we first need to load the tokenizer we want to pair with our model

### NOTE: Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch. This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; our new tokenizer will be exactly the same as facebook/bart-large-mnli, and the only thing that will change is the vocabulary, which will be determined by the training on our corpus.

In [17]:
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)
vocab_size = len(pretrained_tokenizer)
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size)

time: 1.47 s (started: 2023-01-22 22:13:36 +00:00)


In [18]:
print(tokenizer.eos_token_id, ",", tokenizer.vocab_size)

None , 6906
time: 1.25 ms (started: 2023-01-22 22:13:37 +00:00)


In [19]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print(tokens)

[2, 167, 144, 90, 3]
time: 963 µs (started: 2023-01-22 22:13:40 +00:00)


In [20]:
converted = tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 655 µs (started: 2023-01-22 22:13:41 +00:00)


In [21]:
print("pretrained_tokenizer-")
print(pretrained_tokenizer)

print("New tokenizer-")
print(tokenizer)

pretrained_tokenizer-
PreTrainedTokenizerFast(name_or_path='joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
New tokenizer-
PreTrainedTokenizerFast(name_or_path='joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english', vocab_size=6906, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
time: 908 µs (started: 2023-01-22 22:13:41 +00:00)


In [22]:
print("New tokenizer: ", len(tokenizer.tokenize(txt)))
print("Pretrained tokenizer: ", len(pretrained_tokenizer.tokenize(txt)))

New tokenizer:  3
Pretrained tokenizer:  3
time: 1.58 ms (started: 2023-01-22 22:13:49 +00:00)


### Saving tokenizer in local

In [23]:
path=f"{base_path}/"
file_name="Intent-Classification-Commands-balanced-ds-large"
tokenizer.save_pretrained(path+file_name)

('/content/drive/MyDrive/Machine_Learning/Transformers-Classification//Intent-Classification-Commands-balanced-ds-large/tokenizer_config.json',
 '/content/drive/MyDrive/Machine_Learning/Transformers-Classification//Intent-Classification-Commands-balanced-ds-large/special_tokens_map.json',
 '/content/drive/MyDrive/Machine_Learning/Transformers-Classification//Intent-Classification-Commands-balanced-ds-large/vocab.txt',
 '/content/drive/MyDrive/Machine_Learning/Transformers-Classification//Intent-Classification-Commands-balanced-ds-large/added_tokens.json',
 '/content/drive/MyDrive/Machine_Learning/Transformers-Classification//Intent-Classification-Commands-balanced-ds-large/tokenizer.json')

time: 3.65 s (started: 2023-01-22 22:13:56 +00:00)


### Loading tokenizer from local

In [24]:
loaded_tokenizer = AutoTokenizer.from_pretrained(path+file_name)

time: 13.4 ms (started: 2023-01-22 22:14:16 +00:00)


In [25]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)

trained tokenizer: [2, 167, 144, 90, 3]
loaded tokenizer: [2, 167, 144, 90, 3]
time: 3.16 ms (started: 2023-01-22 22:14:17 +00:00)


In [26]:
tokenizer.push_to_hub("dipesh/Intent-Classification-Commands-balanced-ds-large")

CommitInfo(commit_url='https://huggingface.co/dipesh/Intent-Classification-Commands-balanced-ds-large/commit/c6f0365bea325355631a99dee812753b1874ca2c', commit_message='Upload tokenizer', commit_description='', oid='c6f0365bea325355631a99dee812753b1874ca2c', pr_url=None, pr_revision=None, pr_num=None)

time: 6.54 s (started: 2023-01-22 22:14:17 +00:00)


You can then load the tokenizer from anywhere with the from_pretrained() method:

In [27]:
downloaded_tokenizer = AutoTokenizer.from_pretrained("dipesh/Intent-Classification-Commands-balanced-ds-large")

txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained (New) tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)
tokens = downloaded_tokenizer(txt)['input_ids']
print("downloaded tokenizer:", tokens)

Downloading:   0%|          | 0.00/415 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/46.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/151k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

trained (New) tokenizer: [2, 167, 144, 90, 3]
loaded tokenizer: [2, 167, 144, 90, 3]
downloaded tokenizer: [2, 167, 144, 90, 3]
time: 9.49 s (started: 2023-01-22 22:14:24 +00:00)


# PART C: Training a model

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizerFast, RobertaTokenizer
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification, \
    TFRobertaForSequenceClassification, TFAutoModel, AutoModel, AutoModelForSequenceClassification
from sklearn import preprocessing
from datasets import load_dataset

time: 2.45 s (started: 2023-01-22 22:14:33 +00:00)


### Loading Dataset

In [29]:
dataset = load_dataset('dipesh/Intent-Classification-Commands-balanced-ds-large')
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/15311 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1702 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/dipesh___parquet/dipesh--Intent-Classification-Commands-balanced-ds-large-7ae1bbc313ccbd57/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 15311
    })
    validation: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 1702
    })
})

time: 14.6 s (started: 2023-01-22 22:14:36 +00:00)


### Creating Datacollector and evaluator

In [30]:
from transformers import DataCollatorWithPadding
import evaluate


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

time: 3.52 s (started: 2023-01-22 22:14:50 +00:00)


In [31]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

time: 1.3 ms (started: 2023-01-22 22:14:54 +00:00)


### Generating id2label, label2id and num_labels

In [32]:
import json
import pandas as pd

df = pd.read_csv(base_path+dataset_file)
dict_labels = {}
for index, row in df.iterrows():
    if row['labels'] not in dict_labels:
        dict_labels[row['labels']] = row['intent']

# to json
with open('labels.json', 'w') as f:
    json.dump(dict_labels, f)

# load json file
with open('labels.json', 'r') as f:
    dict_labels = json.load(f)


id2label = dict_labels
label2id = {v: int(k) for k, v in dict_labels.items()}
num_labels = len(df.labels.unique())
num_lael = dict_labels
label2id = {v: int(k) for k, v in dict_labels.items()}
num_labels = len(df.labels.unique())
num_labels

107

time: 602 ms (started: 2023-01-22 22:17:08 +00:00)


### Loading Model

In [33]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# If not loaded then load tokenizer by uncommenting below line
# tokenizer = AutoTokenizer.from_pretrained("dipesh/Intent-Classification-Commands-balanced-ds-large")
model = AutoModelForSequenceClassification.from_pretrained(
    base_model, num_labels=num_labels, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

Downloading:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([60, 768]) in the checkpoint and torch.Size([107, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([60]) in the checkpoint and torch.Size([107]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 34.5 s (started: 2023-01-22 22:17:31 +00:00)


### Creating tokenized dataset

In [34]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

time: 870 ms (started: 2023-01-22 22:18:05 +00:00)


In [35]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 15311
    })
    validation: Dataset({
        features: ['text', 'intent', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1702
    })
})

time: 9.77 ms (started: 2023-01-22 22:18:06 +00:00)


### Finetuning Model

In [37]:
config = {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": id2label,
  "initializer_range": 0.02,
  "label2id": label2id,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": False,
  "tie_weights_": True,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

time: 1.58 ms (started: 2023-01-22 22:19:06 +00:00)


In [39]:
with open(path+file_name+'/config.json', "w") as f:
    json.dump(config, f)

time: 13.5 ms (started: 2023-01-22 22:22:35 +00:00)


In [41]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained('joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english')
config.update({"id2label": id2label})
config.update({"label2id": label2id})
# Push the config to your namespace with the name "my-finetuned-bert".
config.push_to_hub("dipesh/distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large")

CommitInfo(commit_url='https://huggingface.co/dipesh/distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large/commit/1a18f7ae5d968d9f1dcc58ad70e0b94ca8dbdc6e', commit_message='Upload config', commit_description='', oid='1a18f7ae5d968d9f1dcc58ad70e0b94ca8dbdc6e', pr_url=None, pr_revision=None, pr_num=None)

time: 5.14 s (started: 2023-01-22 22:31:37 +00:00)


In [42]:
output_dir_model_hf_name="distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large"

training_args = TrainingArguments(
    output_dir=output_dir_model_hf_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/dipesh/distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/256M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Download file runs/Jan21_00-04-33_89782b93478b/events.out.tfevents.1674259476.89782b93478b.15584.2: 100%|#####…

Download file runs/Jan20_23-53-25_89782b93478b/1674258811.4165401/events.out.tfevents.1674258811.89782b93478b.…

...

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, intent. If text, intent are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 15311
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9570
  Number of trainable parameters = 67035755
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,4.440300,3.428414,0.181551
2,3.001600,1.913541,0.598707
3,1.658200,1.171973,0.723854
4,0.959900,0.781283,0.819036
5,0.611700,0.579765,0.865452
6,0.418800,0.513178,0.874266
7,0.292900,0.472325,0.888954
8,0.228900,0.447281,0.894242
9,0.171800,0.436911,0.899530
10,0.145900,0.430507,0.901293


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, intent. If text, intent are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1702
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large/checkpoint-957
Configuration saved in distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large/checkpoint-957/config.json
Model weights saved in distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large/checkpoint-957/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large/checkpoint-957/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large/checkpoint-957/special_tokens_map

TrainOutput(global_step=9570, training_loss=1.1232245586757998, metrics={'train_runtime': 785.9997, 'train_samples_per_second': 194.797, 'train_steps_per_second': 12.176, 'total_flos': 559717548996276.0, 'train_loss': 1.1232245586757998, 'epoch': 10.0})

time: 14min 16s (started: 2023-01-22 22:37:56 +00:00)


# Inference

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=f"./{output_dir_model_hf_name}")

In [59]:
classifier("click photo")

[{'label': 'click_photo', 'score': 0.9945071339607239}]

time: 69 ms (started: 2023-01-22 23:32:37 +00:00)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(
    "dipesh/distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large")
model = AutoModelForSequenceClassification.from_pretrained(
    "dipesh/distilbert-base-uncased-Intent-Classification-Commands-balanced-ds-large")
classifier_return_all_scores = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True )
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [65]:
sequence = "make me laugh"

# Classify the sequence
results = classifier(sequence)

# Print the results
print(results)

[{'label': 'tell_me_joke', 'score': 0.9722713232040405}]
time: 52.5 ms (started: 2023-01-22 23:43:08 +00:00)


In [66]:
sequence = "make me laugh"

# Classify the sequence
results = classifier_return_all_scores(sequence)

# Print the results
print(results)

[[{'label': 'alarm_query', 'score': 0.0003521378093864769}, {'label': 'alarm_remove', 'score': 0.00014693000412080437}, {'label': 'alarm_set', 'score': 6.851143552921712e-05}, {'label': 'asking_date', 'score': 1.101745874620974e-05}, {'label': 'asking_time', 'score': 5.592969500867184e-06}, {'label': 'asking_weather', 'score': 2.59551979979733e-05}, {'label': 'audio_volume_down', 'score': 5.249052628641948e-05}, {'label': 'audio_volume_mute', 'score': 0.00011564674059627578}, {'label': 'audio_volume_up', 'score': 6.199068593559787e-05}, {'label': 'calendar_query', 'score': 7.643279968760908e-05}, {'label': 'calendar_remove', 'score': 8.628347131889313e-05}, {'label': 'calendar_set', 'score': 3.948936864617281e-05}, {'label': 'check_internet_speed', 'score': 0.0008220388554036617}, {'label': 'clever', 'score': 7.532478775829077e-05}, {'label': 'click_photo', 'score': 1.550744127598591e-05}, {'label': 'cooking_recipe', 'score': 0.00011010889284079894}, {'label': 'courtesygoodbye', 'score